# Insert Cursor Demo

- Shows how to use an insert cursor
- Goal: Take a table (csv file) and make it spatial!

In [7]:
import os
from arcpy import da
from arcpy import env
import arcpy

In [8]:
import pandas as pd

In [9]:
# Monument data from a github site. 
df = pd.read_csv("https://raw.githubusercontent.com/mike-n-7/ML4/master/data/monuments.csv")
df.head()

,NO_OUVRAGE,NOM,x,y,LONG,LAT
0,215,CROIX DU MONT-ROYAL,297931.979414,5.040996e+06,-73.587896,45.508851
1,933,MONUMENT LOUIS RUBINSTEIN (PC J.-MANCE),297852.176927,5.041888e+06,-73.588930,45.516872
2,963,MONUMENT FRERE MARIE VICTORIN,300438.173416,5.046376e+06,-73.555870,45.557280
3,1642,LES GRAMINÉES DU JARDIN SAINT-SULPICE,293748.069071,5.046620e+06,-73.641568,45.559403
4,1643,LIMITE TEMPS,292150.196458,5.047910e+06,-73.662069,45.570984


### Create a Feature Class

##### Check if the Feature Class Exists

- recall: When creating feature classes, spatial tables not exist

In [10]:
fc = os.path.join(env.scratchGDB, "monument")
if arcpy.Exists(fc):
    arcpy.management.Delete(fc)

##### Create Spatial Reference

In [11]:
sr = arcpy.SpatialReference(4326)

#### Create The Feature Class 

In [12]:
fc = arcpy.management.CreateFeatureclass(out_name=os.path.basename(fc), 
                                         out_path=os.path.dirname(fc), 
                                         geometry_type="POINT", 
                                         spatial_reference=sr)[0]
print(fc)

C:\Users\lando\AppData\Local\Temp\scratch.gdb\monument


#### Add Fields to the Feature Class

- Use the `arcpy.management.AddField()`

In [13]:
arcpy.management.AddField(in_table=fc, field_name="NOM", field_type="TEXT")
arcpy.management.AddField(in_table=fc, field_name="LONG", field_type="DOUBLE")
arcpy.management.AddField(in_table=fc, field_name="LAT", field_type="DOUBLE")
fields = [fld.name for fld in arcpy.ListFields(fc) \
          if fld.type != "Geometry"]
fields

['OBJECTID', 'NOM', 'LONG', 'LAT']

#### Load the Data

- Use an Insert Cursor to add the information


In [14]:
fields = [fld.name for fld in arcpy.ListFields(fc) \
          if fld.type not in  ["Geometry", "OID"]]
fields += ['Shape@']

In [15]:
fields

['NOM', 'LONG', 'LAT', 'Shape@']

**A Sample of the Data**

In [16]:
dict(zip(df.columns.tolist(), df.values[0].tolist()))

{'NO_OUVRAGE': 215,
 'NOM': 'CROIX DU MONT-ROYAL',
 'x': 297931.97941375,
 'y': 5040996.027761,
 'LONG': -73.587896,
 'LAT': 45.508851}

### Create the Cursor Object

- Use the `with` pattern to create a cursor
- For each row in the Data Frame, **insert** a new row
- Finally clean up the cursor and release the lock
    + `del` ensure the lock is removed (though should not be needed)

In [17]:
with da.InsertCursor(fc, fields) as icur:
    for row in df.values:
        pt = arcpy.Point(row[-2], row[-1])
        ptgeom = arcpy.PointGeometry(pt, sr)
        icur.insertRow([row[1], row[-2], row[-1], ptgeom])
        
del icur

##### Display the Added Rows

In [18]:
arcpy.management.GetCount(fc)[0]

'48'

### Map It

In [19]:
from arcgis.gis import GIS
gis = GIS()

In [20]:
m1 = gis.map()
m1.center = (45.4944962453396,-73.53701983501136)
m1

MapView(layout=Layout(height='400px', width='100%'))

In [21]:
m1.zoom = 12

In [22]:
from arcgis.features import GeoAccessor, GeoSeriesAccessor
sdf = pd.DataFrame.spatial.from_featureclass(fc)
sdf.head()

,OBJECTID,NOM,LONG,LAT,SHAPE
0,1,CROIX DU MONT-ROYAL,-73.587896,45.508851,"{'x': -73.58789599999994, 'y': 45.508851000000..."
1,2,MONUMENT LOUIS RUBINSTEIN (PC J.-MANCE),-73.588930,45.516872,"{'x': -73.58892999999995, 'y': 45.516872000000..."
2,3,MONUMENT FRERE MARIE VICTORIN,-73.555870,45.557280,"{'x': -73.55586999999997, 'y': 45.557280000000..."
3,4,LES GRAMINÉES DU JARDIN SAINT-SULPICE,-73.641568,45.559403,"{'x': -73.64156799999995, 'y': 45.559403000000..."
4,5,LIMITE TEMPS,-73.662069,45.570984,"{'x': -73.66206899999997, 'y': 45.570984000000..."


In [23]:
sdf.spatial.plot(m1)

True